# **Algoritmos Genéricos**

<p style="font-size: smaller; text-align: right;">João Dias & Rafael Rodrigues</p>



## **Objetivo**

Aplicar um Algoritmo Genético (AG) para encontrar o valor máximo da função 
𝑓(𝑥)=𝑥2−3𝑥+4 no intervalo 𝑋=[−10,+10].



## **Estrutura do Notebook**

1. **Introdução**
  
   Explicação sobre os algoritmos genéticos e sua aplicação na otimização de funções.

2. **Representação e População Inicial**

   Codificação dos indivíduos e geração da população inicial.

3. **Função Objetivo**

   Implementação da função 𝑓(𝑥).


4. **Operadores Genéticos**

   Implementação das operações de seleção, crossover e mutação.

5. **Execução do Algoritmo**

   Evolução ao longo de várias gerações e visualização dos resultados.

6. **Conclusão**

   Análise dos resultados e avaliação da eficácia do algoritmo.



## **Introdução**

Os Algoritmos Genéticos (AGs) são meta-heurísticas inspiradas na seleção natural e evolução biológica. Eles utilizam uma população de soluções candidatas que evoluem ao longo de gerações por meio de:

- **Seleção:** Escolha dos melhores indivíduos com base em sua aptidão.
- **Crossover:** Combinação de características entre indivíduos selecionados.
- **Mutação:** Introdução de pequenas variações para manter a diversidade.

Neste notebook, aplicaremos um AG para encontrar a solução que maximiza a função 𝑓(𝑥)no intervalo definido, utilizando:
- População inicial de *4 indivíduos*.
- Taxas de *crossover (70%)* e *mutação (1%)*.
- *Seleção por torneio* como método de escolha.
- Avaliação ao longo de *5 gerações*, podendo ser estendido para *20 gerações*.




---

## **Representação dos indivíduos e População inicial**

Para implementar o Algoritmo Genético (AG), precisamos codificar os valores de 𝑥 como vetores binários. Vamos dividir essa etapa em três partes:

1. *Codificação dos Indivíduos:* Representação de valores de 𝑥 em formato binário.
2. *Decodificação:* Conversão do vetor binário de volta para valores reais no intervalo [−10,10].
3. *Geração da População Inicial:* Criação de uma população inicial de 4 indivíduos.




---

## **1. Configuração Inicial e Funções de Codificação**

In [7]:
import numpy as np
import random

# Parâmetros do problema
NUM_BITS = 8  # Número de bits para representar x
MIN_X = -10   # Limite inferior
MAX_X = 10    # Limite superior

def configurar_ag(taxa_crossover, taxa_mutacao, num_individuos, num_geracoes, seed):
    np.random.seed(seed)
    random.seed(seed)
    return {
        'taxa_crossover': taxa_crossover,
        'taxa_mutacao': taxa_mutacao,
        'num_individuos': num_individuos,
        'num_geracoes': num_geracoes,
        'seed': seed
    }

# Função para gerar um indivíduo aleatório (binário)
def generate_individual(num_bits=NUM_BITS):
    return np.random.randint(0, 2, size=num_bits)  # Vetor binário aleatório

# Função para decodificar um vetor binário para um valor real
def decode_individual(binary_vector, min_x=MIN_X, max_x=MAX_X):
    decimal_value = int("".join(map(str, binary_vector)), 2)  # Converte binário para decimal
    max_value = (2 ** len(binary_vector)) - 1  # Maior valor possível para o número de bits
    x = min_x + (decimal_value / max_value) * (max_x - min_x)
    return x

# Função para gerar a população inicial
def generate_initial_population(pop_size=4, num_bits=NUM_BITS):
    population = [generate_individual(num_bits) for _ in range(pop_size)]
    return population


---

## **2. Função de Fitness**

A função fitness mede o quão “bom” é um indivíduo em resolver o problema. Para este trabalho, o valor de f(x) será a medida de fitness.

Entrada: A população atual (representada em binário).
Decodificação: Converter o vetor binário para valores de x dentro do intervalo [−10,+10].
Cálculo do Fitness: Avaliar f(x) para cada indivíduo.





In [8]:
def binary_to_decimal(binary, lower_bound, upper_bound):
    """
    Converte um vetor binário para um valor decimal no intervalo [lower_bound, upper_bound].
    """
    decimal = int("".join(map(str, binary)), 2)
    max_value = (2 ** len(binary)) - 1
    scaled_value = lower_bound + (decimal / max_value) * (upper_bound - lower_bound)
    return scaled_value

def fitness_function(population, lower_bound=-10, upper_bound=10):
    """
    Calcula o fitness de cada indivíduo na população.
    """
    fitness_values = []
    for individual in population:
        x = binary_to_decimal(individual, lower_bound, upper_bound)
        fitness = x**2 - 3*x + 4  # Função f(x) = x^2 - 3x + 4
        fitness_values.append(fitness)
    return fitness_values



Explicação:

- binary_to_decimal: Converte um vetor binário em um valor decimal escalado para o intervalo desejado ([−10,+10]).

- fitness_function: Calcula o fitness de cada indivíduo da população ao avaliar f(x).

Saída Esperada: Uma lista com os valores de fitness calculados para cada indivíduo.

---

## **3. Seleção por Torneio**

In [9]:
def tournament_selection(population, fitness_values, tournament_size=2):
    """
    Seleção por torneio: seleciona um indivíduo da população com base no fitness.
    """
    selected = []
    for _ in range(len(population)):
        # Escolher aleatoriamente 'tournament_size' indivíduos
        participants = random.sample(list(enumerate(fitness_values)), tournament_size)
        # Encontrar o participante com maior fitness
        winner = max(participants, key=lambda x: x[1])
        selected.append(population[winner[0]])  # Adiciona o indivíduo vencedor
    return selected


---

## **4. Crossover (Recombinação)**

In [10]:
def crossover(parent1, parent2):
    """
    Aplica crossover de um ponto entre dois pais.
    """
    point = random.randint(1, len(parent1) - 1)  # Ponto de corte
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

def apply_crossover(population, crossover_rate=0.7):
    """
    Aplica crossover na população com uma taxa de crossover.
    """
    new_population = []
    for i in range(0, len(population), 2):
        parent1 = population[i]
        parent2 = population[i + 1 if i + 1 < len(population) else 0]  # Garante paridade
        if random.random() < crossover_rate:
            child1, child2 = crossover(parent1, parent2)
        else:
            child1, child2 = parent1, parent2  # Sem crossover
        new_population.extend([child1, child2])
    return new_population


---

## **5. Mutação**

In [11]:
def mutate(individual, mutation_rate=0.01):
    """
    Aplica mutação em um indivíduo com uma taxa definida.
    """
    return [1 - gene if random.random() < mutation_rate else gene for gene in individual]


---

## **6. Execução do Algoritmo**

In [12]:
def run_genetic_algorithm(config, num_generations=5):
    population = generate_initial_population(config['num_individuos'], NUM_BITS)
    best_fitness = -np.inf  # Inicializa o melhor fitness com valor negativo infinitamente grande
    best_individual = None

    for generation in range(num_generations):
        fitness_values = fitness_function(population)
        selected_population = tournament_selection(population, fitness_values)
        population = apply_crossover(selected_population, config['taxa_crossover'])
        population = [mutate(ind, config['taxa_mutacao']) for ind in population]
        
        # Verifica o melhor fitness da geração
        current_best_fitness = max(fitness_values)
        if current_best_fitness > best_fitness:
            best_fitness = current_best_fitness
            best_individual = population[fitness_values.index(current_best_fitness)]
        
        # Exibir informações da geração
        print(f"\nGeração {generation + 1}:")
        for i, individual in enumerate(population):
            x_value = decode_individual(individual)
            fitness = fitness_values[i]
            print(f"Indivíduo {i+1}: x = {x_value:.4f}, Fitness = {fitness:.4f}")

    # Exibir e retornar o melhor indivíduo encontrado após as gerações
    if best_individual is not None:
        best_x_value = decode_individual(best_individual)
        print(f"\nMelhor valor encontrado: x = {best_x_value:.4f}, Fitness = {best_fitness:.4f}")
        
    return best_individual, best_fitness  # Retorna o melhor indivíduo e seu fitness

---